<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTunning_Testing_For_EmotionQADataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Medmcqa.ipynb

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

In [2]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Mistral-7B-v0.1_Emotion"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

In [ ]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt = 'I am going to take a vacation next month.'
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [40]:
outputs

[{'generated_text': 'I am going to take a vacation next month. i feel like i am going to be that person who is going to be very productive while on vacation because i am going to be doing things that i enjoy [/INST] 1 '}]

In [41]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

'i feel like i am going to be that person who is going to be very productive while on vacation because i am going to be doing things that i enjoy [/INST] 1'

In [42]:
def label_classificator(ganswer):
          if int(ganswer) == 0:
            class_label='Sentiment-Label is: Sadness(0)'
            print ()
          if int(ganswer)  == 1:
            class_label='Sentiment-Label is: Joy(1)'
            print ()
          if int(ganswer) == 2:
            class_label='Sentiment-Label is: Love(2)'
            print ()
          if int(ganswer) == 3:
            class_label='Sentiment-Label is: Anger(3)'
            print ()
          if int(ganswer) == 4:
            class_label='Sentiment-Label is: Fear(4)'
            print ()
          if int(ganswer) == 5:
            class_label='Sentiment-Label is: Surprise(5)'
            print ()
          return class_label

In [43]:
prompt = '[INST] I am going to take a vacation next month. [/INST]'
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [44]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

'1'

In [45]:
sentiment=label_classificator(ganswer)
print(f'{sentiment}')


Sentiment-Label is: Joy(1)


In [46]:
prompt = '[INST] I am scared. [/INST]'
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [47]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

'4'

In [48]:
sentiment=label_classificator(ganswer)
print(f'{sentiment}')


Sentiment-Label is: Fear(4)


In [49]:
prompt = '[INST] I grab a minute to post, but I feel greedy and wrong. [/INST]'
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
print(f'{prompt}')
print(f'{ganswer}')
sentiment=label_classificator(ganswer)
print(f'{sentiment}')

[INST] I grab a minute to post, but I feel greedy and wrong. [/INST]
3

Sentiment-Label is: Anger(3)


In [50]:
prompt = 'I grab a minute to post, but I feel greedy and wrong.'

outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
#print(f'{prompt}')
print(f'{ganswer}')

#sentiment=label_classificator(ganswer)
#print(f'{sentiment}')

i don t know. i just want to write that i am feeling very tender and fragile like the petals of a flower that has just begun to bloom [/INST] 2


In [ ]:
from datasets import load_dataset
print("Preprocessing dataset Emotion")
dataset = load_dataset("dair-ai/emotion")

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["validation"].to_json("validation_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")

In [53]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [54]:
nrec=10
dataset_final_Question=dataset['text'][0:nrec]
dataset_final_Answer=dataset['label'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [55]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame

datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [56]:
dataset[6]

{'text': 'i felt anger when at the end of a telephone call', 'label': 3}

In [57]:
datasetF

,Question,Answer
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
5,i cant walk into a shop anywhere where i do no...,4
6,i felt anger when at the end of a telephone call,3
7,i explain why i clung to a relationship with a...,1
8,i like to have the same breathless feeling as ...,1
9,i jest i feel grumpy tired and pre menstrual w...,3


In [59]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")

In [60]:
eval_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [61]:
nrec=6
eval_dataset[nrec]

{'text': 'i felt anger when at the end of a telephone call', 'label': 3}

In [62]:
eval_dataset[nrec]['text']

'i felt anger when at the end of a telephone call'

In [63]:
eval_dataset[nrec]['label']

3

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [64]:
rec=6

In [65]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt =  eval_dataset[rec]['text']
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [66]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

'with a friend the friend didnt reciprocate the same feelings [/INST] 3'

In [67]:
qc=str(ganswer).find('[/INST]')
ganswer=ganswer[qc+8:len(ganswer)]

In [68]:
sentiment=label_classificator(ganswer)
print(f'{sentiment}')


Sentiment-Label is: Anger(3)


In [ ]:
# Inspect the structure of the messages
#print(eval_dataset[rand_idx]["messages"][:2])

#Use the chat template to format the input for the language model
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt = eval_dataset[rec]['text']
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [70]:
outputs

[{'generated_text': 'i felt anger when at the end of a telephone call with a friend the friend didnt reciprocate the same feelings [/INST] 3 '}]

In [71]:
oanswer=eval_dataset[rec]['label']
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
#print(ganswer)
#rand_idx = randint(0, len(eval_dataset))

print(f"Question:\n{eval_dataset[rec]['text']}")
print()
print(f"Original Answer:\n{oanswer}")
print()
print(f"Generated Answer:\n{ganswer}")

from difflib import SequenceMatcher

  # Utility function to compute similarity
def similar(str1, str2):
  return SequenceMatcher(None, str1, str2).ratio()

# using SequenceMatcher.ratio()

qc=str(ganswer).find('[/INST]')
ganswer=ganswer[qc+8:len(ganswer)]

# similarity between strings
similarity = similar(str(oanswer), str(ganswer))

# printing the result
print()
print ("The similarity between 2 strings is : " + str(similarity))

Question:
i felt anger when at the end of a telephone call

Original Answer:
3

Generated Answer:
with a friend the friend didnt reciprocate the same feelings [/INST] 3

The similarity between 2 strings is : 1.0


In [72]:
number_of_eval_samples = 10
eval_dataset.shuffle().select(range(number_of_eval_samples))

Dataset({
    features: ['text', 'label'],
    num_rows: 10
})

In [80]:
from tqdm import tqdm
import re
from difflib import SequenceMatcher

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")
#eval_dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")
#eval_dataset = load_dataset("json", data_files="/content/validation_dataset.json", split="train")

def evaluate(sample):

    prompt = sample['text']
    answer=sample['label']

    if answer == None:
       answer_original = 'Not possible to get or use'

    answer_original = re.sub("\n", "", str(answer))


    if len(answer_original)==0:
        answer_original='Not possible to get or use'

    str_question= sample['text']

    prompt = re.sub("\n", "", str_question)

    #prompt = eval_dataset[rand_idx]['Question']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    #prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    #outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt)+9:].strip()
    #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

    oanswer=sample['label']

    if oanswer == None:
       oanswer = 'Not possible to get or use'

    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
    qc=str(ganswer).find('[/INST]')
    #print(qc)
    if int(qc)<0:
       #print(f"Generated Answer:\n{ganswer}")
       ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
    else:
       ganswer=ganswer[qc+8:len(ganswer)]

    #print(f"Generated Answer:\n{ganswer}")

    # Utility function to compute similarity
    def similar(str1, str2):
      return SequenceMatcher(None, str1, str2).ratio()

    # using SequenceMatcher.ratio()
    # similarity between strings
    #ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()

    res = similar(str(oanswer), str(ganswer))

    # printing the result
    #print ("The similarity between 2 strings is : " + str(res))

    if  res>0.90:
          #print ()
          #print ("The similarity between 2 strings is : " + str(res))
          #print(f"Question:\n{sample['text']}")
          #print(f"Original Answer:\n{answer_original}")
          #print(f"Generated Answer:\n{ganswer}")

          #print ()
          #if int(ganswer) == 0:
          #  print(f'Sentiment-Label is: Sadness(0)')
          #  print ()
          #if int(ganswer)  == 1:
          #  print(f'Sentiment-Label is: Joy(1)')
          #  print ()
          #if int(ganswer) == 2:
          #  print(f'Sentiment-Label is: Love(2)')
          #  print ()
          #if int(ganswer) == 3:
          #  print(f'Sentiment-Label is: Anger(3)')
          #  print ()
          #if int(ganswer) == 4:
          #  print(f'Sentiment-Label is: Fear(4)')
          #  print ()
          #if int(ganswer) == 5:
          #  print(f'Sentiment-Label is: Surprise(5)')
          #  print ()
          return 1
    else:
          #print ()
          #print(f"FULL Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
          #print(f"Question:\n{sample['text']}")
          #print(f"Generated Answer:\n{ganswer}")
          #print(f"Original Answer:\n{oanswer}")
          #print()
          return 0
    del ganswer
success_rate = []
number_of_eval_samples = 2000
# iterate over eval dataset and predict

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
for n in range(number_of_eval_samples):
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

#print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

The following hyperparameters were used during training:

learning_rate: 0.0002 train_batch_size: 3 eval_batch_size: 8 seed: 42 gradient_accumulation_steps: 2 total_train_batch_size: 6 optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08 lr_scheduler_type: constant lr_scheduler_warmup_ratio: 0.03 num_epochs: 1

--------------

NOTE: test - Accuracy (Eval dataset and predict) for a sample of 5: 80.00%

NOTE: validation - Accuracy (Eval dataset and predict) for a sample of 100: 61.00%

NOTE: test - Accuracy (Eval dataset and predict) for a sample of 100: 66.00%

NOTE: test - Accuracy (Eval dataset and predict) for a sample of 1000: 58.90%

NOTE: test - Accuracy (Eval dataset and predict) for a sample of 2000: 59.45%

--------------

The following hyperparameters were used during training:

learning_rate: 0.0002
train_batch_size: 3
eval_batch_size: 8
seed: 42
gradient_accumulation_steps: 2
total_train_batch_size: 6
optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
lr_scheduler_type: constant
lr_scheduler_warmup_ratio: 0.03
num_epochs: 25

---------------

NOTE: test - Accuracy (Eval dataset and predict) for a sample of 2000: 79.95%



The following hyperparameters were used during training:

learning_rate: 0.0002
train_batch_size: 3
eval_batch_size: 8
seed: 42
gradient_accumulation_steps: 2
total_train_batch_size: 6
optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
lr_scheduler_type: constant
lr_scheduler_warmup_ratio: 0.03
num_epochs: 40

---------------

Accuracy (Eval dataset and predict) for a sample of 2000: 80.70%

In [81]:
print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

Accuracy (Eval dataset and predict) for a sample of 2000: 80.70%
